<a id='setup'></a>
#Initial setup 


In [ ]:
# -- MAIN IMPORT

import pyVHR as vhr
import numpy as np

# Plotting: set 'colab' for Google Colaboratory, 'notebook' otherwise
vhr.plot.VisualizeParams.renderer = 'colab'  # or 'notebook'

In [ ]:
# -- LOAD A DATASET

dataset_name = 'ubfc1'                   # the name of the python class handling it 
video_DIR = '/var/datasets/VHR1/UBFC1/'  # dir containing videos
BVP_DIR = '/var/datasets/VHR1/UBFC1/'    # dir containing BVPs GT

dataset = vhr.datasets.datasetFactory(dataset_name, videodataDIR=video_DIR, BVPdataDIR=BVP_DIR)
allvideo = dataset.videoFilenames

# print the list of video names with the progressive index (idx)
for v in range(len(allvideo)):
  print(v, allvideo[v])

In [ ]:
# -- PARAMETER SETTING

wsize = 6           # seconds of video processed (with overlapping) for each estimate 
video_idx = 0      # index of the video to be processed
fname = dataset.getSigFilename(video_idx)
sigGT = dataset.readSigfile(fname)
bpmGT, timesGT = sigGT.getBPM(wsize)
videoFileName = dataset.getVideoFilename(video_idx)
print('Video processed name: ', videoFileName)
fps = vhr.extraction.get_fps(videoFileName)
print('Video frame rate:     ',fps)

In [ ]:
# -- DISPLAY VIDEO FRAMES

vhr.plot.display_video(videoFileName)

# MTTS-CAN

**DeepPhys: Video-Based Physiological Measurement Using Convolutional Attention Networks**

*Weixuan Chen and Daniel McDuff*

**Abstract**. Non-contact video-based physiological measurement has many applications in health care and human-computer interaction. Practical applications require measurements to be accurate even in the presence of large head rotations. We propose the first end-to-end system for video-based measurement of heart and breathing rate using a deep convolutional network. The system features a new motion representation based on a skin reflection model and a new attention mechanism using appearance information to guide motion estimation, both of which enable robust measurement under heterogeneous lighting and major motions. Our approach significantly outperforms all current state-of-the-art methods on both RGB and infrared video datasets. Furthermore, it allows spatial-temporal distributions of physiological signals to be visualized via the attention mechanism.

papers: 
* [DeepPhys: Video-Based Physiological Measurement Using Convolutional Attention Networks](https://web.media.mit.edu/~cvx/docs/18.Chen-etal-ECCV.pdf), 
* [Multi-Task Temporal Shift Attention Networks for On-Device Contactless Vitals Measurement
](https://papers.nips.cc/paper/2020/file/e1228be46de6a0234ac22ded31417bc7-Paper.pdf)


In [ ]:
# extract raw frames
sp = vhr.extraction.sig_processing.SignalProcessing()
frames = sp.extract_raw(videoFileName)
print('Frames shape:', frames.shape)

# apply MTTS_CAN model
bvp_pred = vhr.deepRPPG.MTTS_CAN_deep(frames, fps, verb=1)
bvps = vhr.BPM.BVPsignal(bvp_pred, fps) # BVP object
vhr.plot.visualize_BVPs([bvps.data], 0)

In [ ]:
## -- analysis
from pyVHR.utils.errors import getErrors, printErrors, displayErrors, BVP_windowing
from pyVHR.extraction.utils import sliding_straded_win_idx

# BVP windowing & BPM estimate
bvp_win, timesES = BVP_windowing(bvp_pred, wsize, fps, stride=1)
bpmES = vhr.BPM.BVP_to_BPM(bvp_win, fps) 

# compute and print errors
RMSE, MAE, MAX, PCC, CCC, SNR = vhr.utils.getErrors(bvp_win, fps, bpmES, bpmGT, timesES, timesGT)
vhr.utils.printErrors(RMSE, MAE, MAX, PCC, CCC, SNR)
displayErrors(bpmES, bpmGT, timesES, timesGT)

# Pipeline

In [ ]:
# run on a single video
from pyVHR.analysis.pipeline import DeepPipeline

filename = '/var/datasets/VHR1/UBFC1/after-exercise/vid.avi'

pipe = DeepPipeline()

time, BPM = pipe.run_on_video(filename, method='MTTS_CAN')